# Fundamentos de Rastreamento

### Configuração

Make sure you set your environment variables, including your Google API key.

In [ ]:
# Você pode defini-las diretamente
import os
os.environ["GOOGLE_API_KEY"] = ""
os.environ["LANGSMITH_API_KEY"] = ""
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_PROJECT"] = "langsmith-academy"

In [ ]:
# Ou você pode usar um arquivo .env
from dotenv import load_dotenv
load_dotenv(dotenv_path="../../.env", override=True)

### Rastreamento com @traceable

O decorador @traceable é uma maneira simples de registrar rastreamentos do SDK Python do LangSmith. Simplesmente decore qualquer função com @traceable.

O decorador funciona criando uma árvore de execução para você cada vez que a função é chamada e inserindo-a dentro do rastreamento atual. As entradas da função, nome e outras informações são então transmitidas para o LangSmith. Se a função gerar um erro ou retornar uma resposta, essa informação também é adicionada à árvore, e as atualizações são aplicadas ao LangSmith para que você possa detectar e diagnosticar fontes de erros. Tudo isso é feito em uma thread em segundo plano para evitar bloquear a execução do seu aplicativo.

In [ ]:
# TODO: Importar traceable
import sys
sys.path.append('../../')
from gemini_utils import call_gemini_chat
from typing import List
import nest_asyncio
from utils import get_vector_db_retriever

MODEL_PROVIDER = "google"
MODEL_NAME = "gemini-1.5-flash"
APP_VERSION = 1.0
RAG_SYSTEM_PROMPT = """Você é um assistente para tarefas de perguntas e respostas.
Use as seguintes partes do contexto recuperado para responder à pergunta mais recente na conversa.
Se você não souber a resposta, apenas diga que não sabe.
Use no máximo três frases e mantenha a resposta concisa.
"""

nest_asyncio.apply()
retriever = get_vector_db_retriever()

# TODO: Configurar rastreamento para cada função
def retrieve_documents(question: str):
    return retriever.invoke(question)   # NOTA: Este é um retriever de banco vetorial LangChain, então esta chamada .invoke() será rastreada automaticamente


def generate_response(question: str, documents):
    formatted_docs = "\n\n".join(doc.page_content for doc in documents)
    messages = [
        {
            "role": "system",
            "content": RAG_SYSTEM_PROMPT
        },
        {
            "role": "user",
            "content": f"Contexto: {formatted_docs} \n\n Pergunta: {question}"
        }
    ]
    return call_gemini(messages)


def call_gemini(
    messages: List[dict], model: str = MODEL_NAME, temperature: float = 0.0
) -> str:
    return call_gemini_chat(model, messages, temperature)


def langsmith_rag(question: str):
    documents = retrieve_documents(question)
    response = generate_response(question, documents)
    return response

@traceable cuida do ciclo de vida do RunTree para você!

In [ ]:
question = "Como posso rastrear com o decorador @traceable?"
ai_answer = langsmith_rag(question)
print(ai_answer)

##### Vamos dar uma olhada no LangSmith!

### Adicionando Metadados

O LangSmith suporta o envio de metadados arbitrários junto com rastreamentos.

Metadados são uma coleção de pares chave-valor que podem ser anexados a execuções. Metadados podem ser usados para armazenar informações adicionais sobre uma execução, como a versão da aplicação que gerou a execução, o ambiente no qual a execução foi gerada, ou qualquer outra informação que você queira associar a uma execução. Similar às tags, você pode usar metadados para filtrar execuções na interface do LangSmith, e podem ser usados para agrupar execuções para análise.

In [ ]:
from langsmith import traceable

@traceable(
    # TODO: Adicionar Metadados
    # metadata={"vectordb": "sklearn"}
)
def retrieve_documents(question: str):
    return retriever.invoke(question)

@traceable
def generate_response(question: str, documents):
    formatted_docs = "\n\n".join(doc.page_content for doc in documents)
    messages = [
        {
            "role": "system",
            "content": RAG_SYSTEM_PROMPT
        },
        {
            "role": "user",
            "content": f"Contexto: {formatted_docs} \n\n Pergunta: {question}"
        }
    ]
    return call_gemini(messages)

@traceable(
    # TODO: Adicionar Metadados
    # metadata={"model_name": MODEL_NAME, "model_provider": MODEL_PROVIDER}
)
def call_gemini(
    messages: List[dict], model: str = MODEL_NAME, temperature: float = 0.0
) -> str:
    return call_gemini_chat(model, messages, temperature)

@traceable
def langsmith_rag(question: str):
    documents = retrieve_documents(question)
    response = generate_response(question, documents)
    return response

In [ ]:
question = "Como adiciono Metadados a uma Execução com @traceable?"
ai_answer = langsmith_rag(question)
print(ai_answer)

Você também pode adicionar metadados em tempo de execução!

In [ ]:
question = "Como adiciono metadados em tempo de execução?"
ai_answer = langsmith_rag(question, langsmith_extra={"metadata": {"runtime_metadata": "foo"}})
print(ai_answer)

##### Vamos dar uma olhada no LangSmith!